# Transfer Capsnet OCT classifier - Arjun Balaji

In [25]:
import numpy as np
import torch, os, sys, time
import model as m
import dataset as d
from options import OptionsHome
from oct_dataset import RandomCrop, get_image

In [27]:
opts = o.OptionsHome()
#opts.parse()
#opts.start_size

In [21]:
class RandomSingleImageCrop(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size
        
        
    def __call__(self, image):

        
        h, w, _ = image.shape
        
        new_h, new_w = self.output_size
        
        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)
        
        image = image[top:top + new_h, left:left + new_w, :]
        #label = label[top:top + new_h, left:left + new_w, :]
    
        return image#, label
    

below is prelim octclassification dataset 

labels_dict is a dict {'name':hard or not?} if hard [0,1] if not [1,0] REMEMBER TO CLASS IMBALANCE HERE

keys of label_dict become possible names for this dataset

In [22]:
class OCTClassificationDataset(Dataset):
    """
    First we create a dataset that will encapsulate our data. It has 3 special 
    functions which will be explained as they go. We will pass this dataset object
    to the torch dataloader object later which will make training easier.
    """
    def __init__ (self,
                  image_data_dir,
                  labels_dict,
                  start_size,
                  cropped_size,
                  transform):
        self.image_data_dir = image_data_dir
        self.labels_dir = labels_dict 
        self.start_size = start_size
        self.transform = transform
        self.cropped_size = cropped_size
        
        
        self.rcrop = RandomSingleImageCrop(self.cropped_size)
        self.phflip = np.random.rand()
        self.pvflip = np.random.rand()
        
        #iterate through the 2d images and get all their name
            
        self.name_list = list(self.labels_dict.keys())
    
    def visualise(self, idx):
        
        sample = self.__getitem__(idx)
        #print(sample['input'].size())
        #print(sample['label'].size())
        input_data = sample['input'].cpu().numpy()[0,:,:]
        label = sample['label'].cpu().numpy()[0]

        """
        put control flow to get easy or hard depdending on label
        """
        
        
        f, (axin, axl, ax1comb) = plt.subplots(1,3, sharey=True)
        f.subplots_adjust(hspace=0.3)
        plt.tight_layout()
        
        #plot image
        image = axin.imshow(input_data,
                            aspect = 'equal')
        f.colorbar(image, ax=axin, orientation='vertical', fraction = 0.05)
        
        axl.imshow(l_data,
                   aspect = 'equal')
        
        
        combined = input_data + 1 * l_data 
        
        ax1comb.imshow(combined, aspect = 'equal')
        plt.show()
        
    def __getitem__(self, idx):
        """This function will allow us to index the data object and it will 
        return a sample."""
        name = self.name_list[idx]
        
        #load data  
        label = self.labels_dict[name]
        
        image = get_image(self.image_data_dir, name, 'images')
        image = image.astype(float)
        
        
        #print(image.shape)
        
        image = np.transpose(image, (1, 2, 0))
        #print(label.max())
        #print(Image.shape)
        if self.transform:
            
            ysize = self.start_size[0] + 20
            xsize = self.start_size[1] + 20
            image = skitransforms.resize(image, output_shape=(ysize, xsize))          
            
            #print(label.shape)
            #print(label.max())
            image = self.rcrop(image)
            #print(label.max())
            
            if self.phflip>0.5:
                #hflip
                image = np.flip(image, 1)
                #print(label.max())
            #print(label.shape)
            
            if self.pvflip>0.5:
                #vflip
                image = np.flip(image, 0)
                #print(label.max())
            #print(label.shape)
            
            angle = np.random.randint(0,360)
            image = skitransforms.rotate(image, angle=angle, mode='reflect')
            #print(label.max())
            #print(label.shape)
            
            if np.random.rand() > 0.5:
                image = gaussian(image, sigma=1, mode='reflect')
            
            
        else:
            image = skitransforms.resize(image, output_shape= self.start_size)
        
        #image = np.expand_dims(preprocessing.scale(image[:,:,0]), -1)
        
        image = np.transpose(image.copy(), (2, 0, 1))
        #og = preprocessing.MinMaxScaler(og)
        
        sample = {'input': torch.tensor(image),
                  'label': torch.tensor(label),
                  'case_name': name}

        return sample
    
    def __len__(self):    
        """This function is mandated by Pytorch and allows us to see how many 
        data points we have in our dataset"""
        return len(self.name_list)
    

init oct data set here!

In [34]:
octdata = OCTClassificationDataset(blah blah blah!)

SyntaxError: invalid syntax (<ipython-input-34-9a4cd739bdcc>, line 1)